Analysis on scraped dataset

## Import

In [0]:
import pandas as pd
import numpy as np

In [0]:
import pickle
import sys
import nltk
from nltk.stem.porter import *

from sklearn.feature_extraction.text import TfidfVectorizer
import matplotlib.pyplot as plt
import seaborn
%matplotlib inline

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
cf_data_1 = pd.read_csv('/content/drive/My Drive/Hate_Speech_Detection_git/data_1/hatespeech_NAACL_SRW.csv',encoding = "ISO-8859-1")
cf_data_2 = pd.read_csv('/content/drive/My Drive/Hate_Speech_Detection_git/data_1/hatespeech_NLP+CSS.csv')

## this is the scraped data

In [0]:

labels_1 = pd.read_csv('/content/drive/My Drive/Hate_Speech_Detection_git/data_1/NAACL_SRW_2016.csv',header=None,names=['ID','class'])
labels_2 = pd.read_csv('/content/drive/My Drive/Hate_Speech_Detection_git/data_1/NLP+CSS_2016.csv',sep='\s')

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  


In [0]:
labels_2.rename_axis({'TweetID':'ID','Expert':'class'},axis=1,inplace=True)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: FutureWarning: Using 'rename_axis' to alter labels is deprecated. Use '.rename' instead
  """Entry point for launching an IPython kernel.


### First lets explore cf_data_1

In [0]:
cf_data_1.rename_axis({'Unnamed: 0':'index_col'},axis=1,inplace=True)
cf_data_2.rename_axis({'Unnamed: 0':'index_col'},axis=1,inplace=True)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: FutureWarning: Using 'rename_axis' to alter labels is deprecated. Use '.rename' instead
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: FutureWarning: Using 'rename_axis' to alter labels is deprecated. Use '.rename' instead
  


## Basic Data Analysis

#### Merging the cf_data_1 with its labelss

In [0]:
labels_1 = labels_2.copy()
cf_data_1 = cf_data_2.copy()

In [0]:
labels_1['ID'] = labels_1['ID'].astype(int)
labels_1['ID'].nunique()
## unique IDs in label file

6909

In [0]:
print('Null IDs in data 1 = ' ,cf_data_1['ID'].isna().sum())

Null IDs in data 1 =  0


In [0]:
(cf_data_1['ID']==0).sum()

0

No zeros in ID so lets just fill NaN IDs with 0 so that we can convert our rest of the IDs into integer

In [0]:
cf_data_1['ID'].fillna(0,inplace=True)
cf_data_1['ID'] = cf_data_1['ID'].astype(int)

In [0]:
print('data 1 shape ='  ,cf_data_1.shape)
print('IDs common in data 1 and labels_1 =',sum(cf_data_1['ID'].isin(labels_1['ID'])))

data 1 shape = (6271, 11)
IDs common in data 1 and labels_1 = 6271


##### Means we couldnt get a few tweets

##### Merging both to assign class label to each tweet

In [0]:
train_1 = cf_data_1.merge(labels_1, on='ID',how='inner')#['class'].isna().sum()

### Function for merging

#### There are no NAs getting generated from labels of NLP+CSS, but some problem with the NAACL, there are two dataponts which dont make any sense, seems like the data collection has got some error there, but lite max, that was generated by the guthub repo author so wont be much wrong, the other file was generated by us which seems legit. Lets explore some of the datapoints of NAACL manually later

#### Okay so basically a few IDs are occuring more than once in the data, which is not good, we should remove such examples, as there are not many of them, only IDs are there which occuring multiple times in data

### Target Analysis

In [0]:
train_1['class'].value_counts()

neither    5526
sexism      639
racism       80
both         26
Name: class, dtype: int64

In [0]:
train = train_1[pd.notnull(train_1['class'])].copy()

In [0]:
train.reset_index(drop=True,inplace=True)

### Very basic exploration

In [0]:
train.rename(columns={'Tweets':'tweet'},inplace=True)

In [0]:
train['tweet'] = train['tweet'].astype(str)

In [0]:
## word and char count, avg_word length and all
train['word_count'] = train['tweet'].apply(lambda x: len(str(x).split(" ")))
train[['tweet','word_count']].head()

,tweet,word_count
0,@OutworldDOTA2 i'm very entertained that all i...,20
1,Indian women want to watch a graphical rape do...,17
2,RT @cage_rusty: #BlameOneNotAll but apparently...,23
3,Sex is fun. Money is rad. People are weird.,9
4,@dylanw i didn't come at this looking to attac...,19


In [0]:
train['char_count'] = train['tweet'].str.len() ## this also includes spaces
train[['tweet','char_count']].head()

,tweet,char_count
0,@OutworldDOTA2 i'm very entertained that all i...,114
1,Indian women want to watch a graphical rape do...,103
2,RT @cage_rusty: #BlameOneNotAll but apparently...,140
3,Sex is fun. Money is rad. People are weird.,43
4,@dylanw i didn't come at this looking to attac...,91


In [0]:
def avg_word(sentence):
    words = sentence.split()
    return (sum(len(word) for word in words)/len(words))

train['avg_word'] = train['tweet'].apply(lambda x: avg_word(x))
train[['tweet','avg_word']].head()

,tweet,avg_word
0,@OutworldDOTA2 i'm very entertained that all i...,4.750000
1,Indian women want to watch a graphical rape do...,5.117647
2,RT @cage_rusty: #BlameOneNotAll but apparently...,5.130435
3,Sex is fun. Money is rad. People are weird.,3.888889
4,@dylanw i didn't come at this looking to attac...,3.842105


#### Hashtags

In [0]:
train['hastags'] = train['tweet'].apply(lambda x: len([x for x in x.split() if x.startswith('#')]))
train[['tweet','hastags']].head()

,tweet,hastags
0,@OutworldDOTA2 i'm very entertained that all i...,0
1,Indian women want to watch a graphical rape do...,2
2,RT @cage_rusty: #BlameOneNotAll but apparently...,1
3,Sex is fun. Money is rad. People are weird.,0
4,@dylanw i didn't come at this looking to attac...,0


### Basic Preprocessing

In [0]:
train['tweet']

0       @OutworldDOTA2 i'm very entertained that all i...
1       Indian women want to watch a graphical rape do...
2       RT @cage_rusty: #BlameOneNotAll but apparently...
3             Sex is fun. Money is rad. People are weird.
4       @dylanw i didn't come at this looking to attac...
5       RT @PeerWorker: @freebsdgirl You just lost $10...
6       @wrarebit what does that have to do with anyth...
7       @alex or in properly angled photographs at low...
8       take off your VC fedoras and think about what ...
9                                      Oh, no.  My inbox.
10      @SAOnline_HF @colepram @ThatAwesomeTerr why ar...
11                                     @composerose yeah.
12      @Petterwass @voretaq7 I also read it as a gay ...
13      It is 2015. Java's installer still prompts to ...
14      @NoraReed @KivaBay @tonitonirocca this. i love...
15      @nat_com1 @Feminazi_Front We know there is a b...
16                             @dtm609 jira/bitbucket. :)
17            

Every word followed by @ is some twitter ID of an user, which shouldn't be considered in our analysis, so lets do the stemming, where we remove @ alonwith the word followed by it

#### Removing user IDs from text

In [0]:
train['tweet'] = train['tweet'].apply(lambda x:' '.join(i for i in [a for a in x.split() if a.find('@')==-1]))

#### Converting to Lower case

In [0]:
train['tweet'] = train['tweet'].apply(lambda x: " ".join(x.lower() for x in x.split()))
train['tweet'].head()

0    i'm very entertained that all it takes is "155...
1    indian women want to watch a graphical rape do...
2    rt #blameonenotall but apparently public space...
3          sex is fun. money is rad. people are weird.
4    i didn't come at this looking to attack him. h...
Name: tweet, dtype: object

There is no use of punctuation mostly, so removing it

In [0]:
train['tweet'] = train['tweet'].str.replace('[^\w\s]','')
train['tweet'].head()

0    im very entertained that all it takes is 155 i...
1    indian women want to watch a graphical rape do...
2    rt blameonenotall but apparently public spaces...
3             sex is fun money is rad people are weird
4    i didnt come at this looking to attack him hes...
Name: tweet, dtype: object

#### Removing stopwords

In [0]:
# import nltk
# nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [0]:
from nltk.corpus import stopwords
stop = stopwords.words('english')
train['tweet'] = train['tweet'].apply(lambda x: " ".join(x for x in x.split() if x not in stop))
train['tweet'].head()

0    im entertained takes 155 iq know precisely dis...
1    indian women want watch graphical rape docu ca...
2    rt blameonenotall apparently public spaces dan...
3                       sex fun money rad people weird
4          didnt come looking attack hes one told piss
Name: tweet, dtype: object

#### Most common and rare words

In [0]:
freq = pd.Series(' '.join(train['tweet']).split()).value_counts()#[:10]
freq

rt                    658
im                    567
feminazi              533
dont                  368
like                  358
people                331
mkr                   310
get                   258
women                 212
know                  211
think                 208
one                   206
ive                   203
would                 189
time                  187
p                     186
really                184
amp                   176
going                 173
coon                  160
oh                    154
need                  154
want                  150
blameonenotall        150
thats                 148
good                  147
see                   147
twitter               145
got                   145
youre                 141
                     ... 
draft                   1
coach                   1
msn                     1
brainsz                 1
crashed                 1
needles                 1
locking                 1
flee        

rt is a very frequent word occuring almost in every tweet, and it doesnt hold any meaning so lets just remove it, I guess it there are such more words but those are all seem to be hatespeech. So we will just keep hate speechwords from above list, lets remove the rest

In [0]:
remove_word = ['rt','mkr','im','dont','like','people','get','think','would','cant']

In [0]:
train['tweet'] = train['tweet'].apply(lambda x: " ".join(x for x in x.split() if x not in remove_word))
train['tweet'].head()

0    entertained takes 155 iq know precisely discussed
1    indian women want watch graphical rape docu ta...
2    blameonenotall apparently public spaces danger...
3                              sex fun money rad weird
4          didnt come looking attack hes one told piss
Name: tweet, dtype: object

In [0]:
freq = pd.Series(' '.join(train['tweet']).split()).value_counts()#[-20000:]
freq

feminazi              533
women                 212
know                  211
one                   206
ive                   203
time                  187
p                     186
really                184
amp                   176
going                 173
coon                  160
oh                    154
need                  154
blameonenotall        150
want                  150
thats                 148
good                  147
see                   147
got                   145
twitter               145
youre                 141
still                 134
even                  127
things                124
gamergate             123
someone               122
well                  122
pretty                121
go                    121
make                  117
                     ... 
coach                   1
msn                     1
brainsz                 1
idiotic                 1
host                    1
crashed                 1
needles                 1
locking     

In [0]:
freq.shape

(10492,)

Doesnt really make sense to remove rare words, i.e. the words with count 1. Because we might lose hateful words this way

In [0]:
from textblob import TextBlob
# nltk.download('punkt')
TextBlob(train['tweet'][1]).words

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


WordList(['indian', 'women', 'want', 'watch', 'graphical', 'rape', 'docu', 'take', 'sexist', 'joke', 'feminazi', 'adarshliberal'])

#### Lemmetization
smart stemming

In [0]:
from textblob import Word
# nltk.download('wordnet')
train['tweet'] = train['tweet'].apply(lambda x: " ".join([Word(word).lemmatize() for word in x.split()]))
train['tweet'].head()

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


0     entertained take 155 iq know precisely discussed
1    indian woman want watch graphical rape docu ta...
2    blameonenotall apparently public space dangero...
3                              sex fun money rad weird
4           didnt come looking attack he one told piss
Name: tweet, dtype: object

##### So now it seems that the data is almost clean! Lets explore advanced preprocessing/ text representation techniques

### Target creation

In [0]:
train['class'].unique()#.isna().sum()

array(['neither', 'sexism', 'racism', 'both'], dtype=object)

In [0]:
# train['class'].replace(['racism', 'none', 'sexism'],[1,0,1],inplace=True)
# train['class'].value_counts()
train['class'].replace(['neither', 'sexism', 'racism', 'both'],[0,1,1,1],inplace=True)
train['class'].value_counts()

0    5526
1     745
Name: class, dtype: int64

### N-Grams

In [0]:
TextBlob(train['tweet'][0]).ngrams(2)

[WordList(['entertained', 'take']),
 WordList(['take', '155']),
 WordList(['155', 'iq']),
 WordList(['iq', 'know']),
 WordList(['know', 'precisely']),
 WordList(['precisely', 'discussed'])]

### TF-IDF

In [0]:
from sklearn.model_selection import train_test_split
x_t,x_v,y_t,y_v = train_test_split(train['tweet'],train['class'],test_size=0.2,random_state=234)

In [0]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer(max_features=1000, lowercase=True, analyzer='word',
 stop_words= 'english',ngram_range=(1,1))

tfidf.fit(train['tweet'])

x_t = tfidf.transform(x_t)
x_v = tfidf.transform(x_v)


In [0]:
x_t.todense().shape

(5016, 1000)

In [0]:
1 - y_v.mean()

0.8780876494023905

In [0]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier

In [0]:
# clf = GradientBoostingClassifier(learning_rate=0.1, max_depth=10)
# clf.fit(x_t,y_t)

GradientBoostingClassifier(criterion='friedman_mse', init=None,
                           learning_rate=0.1, loss='deviance', max_depth=10,
                           max_features=None, max_leaf_nodes=None,
                           min_impurity_decrease=0.0, min_impurity_split=None,
                           min_samples_leaf=1, min_samples_split=2,
                           min_weight_fraction_leaf=0.0, n_estimators=100,
                           n_iter_no_change=None, presort='auto',
                           random_state=None, subsample=1.0, tol=0.0001,
                           validation_fraction=0.1, verbose=0,
                           warm_start=False)

In [0]:
def model_training(clf, x_t, y_t, x_v=None , y_v=None ,task='binary:logistic'):
    clf.fit(x_t,y_t)
    print('training accuracy', clf.score(x_t,y_t))
    
    if task=='binary:logistic':
        # if x_v==None:
        #   pass

        # else:
          print('validation accuracy', clf.score(x_v,y_v))
          print('validation f1_score',f1_score(clf.predict(x_v),y_v))
          print('validation roc_auc score',roc_auc_score(y_v,clf.predict_proba(x_v)[::,-1]))
          print('confusion matrix \n',confusion_matrix(y_v, clf.predict(x_v)))
    
    if task=='reg:linear':
        if x_v==None:
          pass
        
        else:
            print('validation r2_score', clf.score(x_v,y_v))
            print('validation MSE',mean_squared_error(clf.predict(x_v),y_v))

            
    return clf

In [0]:
clf = GradientBoostingClassifier(learning_rate=0.1, max_depth=10)
model_training(clf,x_t,y_t,x_v,y_v)

training accuracy 0.9876395534290271
validation accuracy 0.9203187250996016
validation f1_score 0.6527777777777779
validation roc_auc score 0.8923140339015219
confusion matrix 
 [[1061   41]
 [  59   94]]


GradientBoostingClassifier(criterion='friedman_mse', init=None,
                           learning_rate=0.1, loss='deviance', max_depth=10,
                           max_features=None, max_leaf_nodes=None,
                           min_impurity_decrease=0.0, min_impurity_split=None,
                           min_samples_leaf=1, min_samples_split=2,
                           min_weight_fraction_leaf=0.0, n_estimators=100,
                           n_iter_no_change=None, presort='auto',
                           random_state=None, subsample=1.0, tol=0.0001,
                           validation_fraction=0.1, verbose=0,
                           warm_start=False)

In [0]:
clf.score(x_t,y_t)

0.988237639553429

In [0]:
clf.score(x_v,y_v)

0.9171314741035856

In [0]:
truth_check = (clf.predict(x_v)==y_v)

In [0]:
train.iloc[truth_check[truth_check==True].index]['tweet']

1475     love phrase fuckboy pain thank adding vocabulary
2905    ive seen support place didnt expect originally...
1454    kickstarter tempting weird money still feel ki...
2816                                             need sit
2609                      want 5 private repos github gdi
5072                            eye makeup make difficult
623           attack black men hate doesnt even know coon
228                                     feminazi detected
4177    comment might helping pas judgement anyones ac...
5877    friend need help knowing talk police preventio...
3989                   clear dog named leonard nimoy llap
613     fascism common sense nationalism kind leftard ...
3820                                                  who
271                        easier github time spend admin
3121    able criticize long isnt abusive may argue cri...
337     lush vanilla powder also make good dry conditi...
1586                                              oh dear
3787          

In [0]:
train.iloc[truth_check[truth_check==False].index]['tweet']

4530    blameonenotall men hurt woman actually need co...
5774       otter creek redneck fat coon httptcomdkjyhfcaz
5893    oh see sending little harassment squad see rig...
2540    blameonenotall want congratulated rapist wanna...
2743    werent kidding said deconstructed yeh seen bet...
3703    cannot stop looking nikkis dreadful black croo...
6157    blocked alexa pro dick breastplate pow equalit...
4435    m bardot see happening boldly declares marxist...
2453    hot single area gtgo west hot single circling ...
1472    thats sas plate still start though amp find th...
6244    httptcoye7qrymd8y gamergate kristi think anita...
3708    sure give good one bad youre probably ugly dir...
1466    ah yes evil set pa getting sjw coffee helping ...
6020      katie nikki pete evans everything wrong society
2707    feminist feminism however want judge feminism ...
6230    lol dumb liberal cultural marxist feminazi ful...
3578    blameonenotall repugnant allows ci het male op...
4128          

In [0]:
from sklearn.metrics import f1_score, classification_report, roc_auc_score

In [0]:
f1_score(clf.predict(x_v),y_v)

0.6462585034013606

In [0]:
print(classification_report(clf.predict(x_v),y_v))

              precision    recall  f1-score   support

           0       0.96      0.95      0.95      1114
           1       0.62      0.67      0.65       141

    accuracy                           0.92      1255
   macro avg       0.79      0.81      0.80      1255
weighted avg       0.92      0.92      0.92      1255



## Word2vec

In [0]:
import gensim
import logging

In [0]:
from gensim.models import Word2Vec

In [0]:
wv = gensim.models.KeyedVectors.load_word2vec_format("/content/drive/My Drive/Hate_Speech_Detection_git/GoogleNews-vectors-negative300.bin.gz", binary=True)
wv.init_sims(replace=True)

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [0]:
def word_averaging(wv, words):
    all_words, mean = set(), []
    
    for word in words:
        if isinstance(word, np.ndarray):
            mean.append(word)
        elif word in wv.vocab:
            mean.append(wv.syn0norm[wv.vocab[word].index])
            all_words.add(wv.vocab[word].index)

    if not mean:
        logging.warning("cannot compute similarity with no input %s", words)
        # FIXME: remove these examples in pre-processing
        return np.zeros(wv.vector_size,)

    mean = gensim.matutils.unitvec(np.array(mean).mean(axis=0)).astype(np.float32)
    return mean

def  word_averaging_list(wv, text_list):
    return np.vstack([word_averaging(wv, post) for post in text_list ])

In [0]:
def w2v_tokenize_text(text):
    tokens = []
    for sent in nltk.sent_tokenize(text, language='english'):
        for word in nltk.word_tokenize(sent, language='english'):
            if len(word) < 2:
                continue
            tokens.append(word)
    return tokens
    
train_w2v, test_w2v = train_test_split(train, test_size=0.2, random_state = 42)
# x_t,x_v,y_t,y_v = train_test_split(train['tweet'],train['class'],test_size=0.2,random_state=234)

test_tokenized = test_w2v.apply(lambda r: w2v_tokenize_text(r['tweet']), axis=1).values
train_tokenized = train_w2v.apply(lambda r: w2v_tokenize_text(r['tweet']), axis=1).values

X_train_word_average = word_averaging_list(wv,train_tokenized)
X_test_word_average = word_averaging_list(wv,test_tokenized)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: DeprecationWarning: Call to deprecated `syn0norm` (Attribute will be removed in 4.0.0, use self.wv.vectors_norm instead).
  
/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


In [0]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import f1_score, classification_report, roc_auc_score, confusion_matrix, accuracy_score



In [83]:
train_w2v['class'].value_counts()

0    4429
1     587
Name: class, dtype: int64

In [84]:
test_w2v['class'].value_counts()s

0    1097
1     158
Name: class, dtype: int64

In [80]:
logreg = LogisticRegression(n_jobs=1, C=1e5)
logreg = logreg.fit(X_train_word_average, train_w2v['class'])
y_pred = logreg.predict(X_test_word_average)
print('accuracy %s' % accuracy_score(y_pred, test_w2v['class']))
print(classification_report(test_w2v['class'], y_pred))
print(confusion_matrix(test_w2v['class'],y_pred))

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


accuracy 0.9155378486055777
              precision    recall  f1-score   support

           0       0.93      0.98      0.95      1097
           1       0.75      0.50      0.60       158

    accuracy                           0.92      1255
   macro avg       0.84      0.74      0.78      1255
weighted avg       0.91      0.92      0.91      1255

[[1070   27]
 [  79   79]]


In [0]:
clf = GradientBoostingClassifier(learning_rate=0.1, max_depth=10)
model_training(clf,X_train_word_average,train_w2v['class'],X_test_word_average,test_w2v['class'],)

training accuracy 0.9964114832535885
validation accuracy 0.9123505976095617
validation f1_score 0.5217391304347827
validation roc_auc score 0.8792766232417525
confusion matrix 
 [[1085   12]
 [  98   60]]


GradientBoostingClassifier(criterion='friedman_mse', init=None,
                           learning_rate=0.1, loss='deviance', max_depth=10,
                           max_features=None, max_leaf_nodes=None,
                           min_impurity_decrease=0.0, min_impurity_split=None,
                           min_samples_leaf=1, min_samples_split=2,
                           min_weight_fraction_leaf=0.0, n_estimators=100,
                           n_iter_no_change=None, presort='auto',
                           random_state=None, subsample=1.0, tol=0.0001,
                           validation_fraction=0.1, verbose=0,
                           warm_start=False)